# Using Timelink in notebooks: tutorial

> First time use: follow instructions in the [README.md](README.md) file in this directory.

In [2]:
import timelink
timelink.version

'1.1.15'

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [3]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()



TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

In [4]:
tlnb.print_info()

Timelink version: 1.1.15
Project name: tutorial
Project home: /workspaces/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: col1V...
Kleio server URL: http://127.0.0.1:8091
Kleio server home: /workspaces/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: naughty_ptolemy
Kleio version requested: latest
Kleio server version: 12.6.577 (2024-10-24 16:53:53)
SQLite directory: /workspaces/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/workspaces/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Database status

Count the number of rows in each table in the database.


In [7]:
tlnb.table_row_count_df()

,table,count
0,acts,39
1,acusacoes,0
2,adendas,0
3,alembic_version,1
4,aregisters,0
5,attributes,2412
6,blinks,4
7,cartas,0
8,casos,0
9,class_attributes,63


### Show the kleio files available 

In [10]:
kleio_files = tlnb.get_kleio_files(recurse=True)
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/auc-alunos.cli,auc-alunos.cli,2025-01-10 08:07:08.567512+00:00,T,2024-05-02 23:34:00+00:00,0,0,N,0,0,None,None,NaT,/rest/reports/sources/auc-alunos.rpt,/rest/exports/sources/auc-alunos.xml
1,sources/b1685.cli,b1685.cli,2025-01-10 07:07:03.744927+00:00,W,2025-01-10 07:07:00+00:00,0,1,I,0,0,No errors,No warnings,2025-01-10 07:07:38.557437,/rest/reports/sources/b1685.rpt,/rest/exports/sources/b1685.xml
2,sources/dehergne-a.cli,dehergne-a.cli,2025-01-10 07:07:07.206591+00:00,V,2025-01-10 07:07:00+00:00,0,0,E,3,0,ERROR: dehergne-a.cli processing same_as relat...,No warnings,2025-01-10 07:07:19.935599,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
3,sources/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2025-01-10 07:07:05.549587+00:00,V,2025-01-10 07:07:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-10 07:07:26.982938,/rest/reports/sources/dehergne-locations-1644.rpt,/rest/exports/sources/dehergne-locations-1644.xml
4,sources/real-entities/real-entities.cli,real-entities.cli,2025-01-10 07:07:05.796856+00:00,V,2025-01-10 07:07:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-10 07:07:31.982903,/rest/reports/sources/real-entities/real-entit...,/rest/exports/sources/real-entities/real-entit...


## Optionally clean translations 

In [9]:
# tlnb.kleio_server.translation_clean("", recurse="yes")


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [12]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

INFO:root:Request translation of T sources/auc-alunos.cli
INFO:root:Importing sources/auc-alunos.cli


## Check the status of the files

Check the import status of the translated files

I: Imported

E: Imported with error

W: Imported with warnings no errors

N: Not imported

U: Translation updated need to reimport

In [11]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,b1685.cli,0,1,kleio/b1685.cli
1,E,3,3,dehergne-a.cli,0,0,kleio/dehergne-a.cli
2,I,0,0,dehergne-locations-1644.cli,0,0,kleio/dehergne-locations-1644.cli
3,I,0,0,real-entities.cli,0,0,kleio/real-entities/real-entities.cli


### Check the translation report

In [12]:
rpt = tlnb.get_translation_report(imported_files_df, 2)
print(rpt)


KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
4-12-2024 9-3

Processing data file dehergne-locations-1644.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 57
Obs: 
** Processing source fonte$dehergne-locations-1644
97: geodesc$deh-chre-1644
*** End of File


Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/local/timelink/clio/src/gacto2.str.json

Kleio file: /kleio-home/kleio/dehergne-locations-1644.cli
Original file: /kleio-home/kleio/dehergne-locations-1644.org
Previous version: /kleio-home/kleio/dehergne-locations-1644.old
Temp file with ids: /kleio-home/kleio/dehergne-locations-1644.ids
** - /kleio-home/kleio/dehergne-locations-1644.cli-renamed to- /kl

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [13]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

b1685.cli
No errors
No warnings




## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,paths=List[str], rows=List[int],status="T"])
* TimelinkNotebook.import([files_df,paths=List[str], rows=List[int],status="V"])



# Obter dados
---

# Getting data

### Procurar pessoa, mostrar em notação Kleio

---
### Search for people, show in Kleio notation

In [15]:
from timelink.api.models import Person

show_only=10

with tlnb.db.session() as session:
    persons = session.query(Person).filter(Person.name.like('%Andrade')).all()

    for person in persons[:show_only]:
        print(person.to_kleio())
        print()

n$António de Andrade/m/id=deh-antonio-de-andrade
  rel$function-in-act/n/lista/dehergne-notices-a//obs=line: 881
  ls$nacionalidade/Portugal/
  ls$jesuita-estatuto/Padre/
  ls$estadia@wikidata/https://www.wikidata.org/wiki/Q668%Índia/
  ls$estadia/Índia#@wikidata:Q668/
  ls$estadia@wikidata/https://www.wikidata.org/wiki/Q42941%Agra/
  ls$estadia/Agra#@wikidata:Q42941/
  ls$jesuita-cargo/Visitador%envoyé comme visiteur et supérieur à Agra/
  ls$jesuita-cargo/Superior%envoyé comme visiteur et supérieur à Agra/
  ls$estadia@wikidata/https://www.wikidata.org/wiki/Q5451%Himalaias/
  ls$estadia/Himalaias#o primeiro Europeu a visitar os Himalaias @wikidata:Q5451/
  ls$estadia@wikidata/https://www.wikidata.org/wiki/Q2658740%Tsaparang, Tibete/
  ls$estadia/Tsaparang, Tibete#@wikidata:Q2658740/
  ls$dehergne/39//obs=Andrade, António de (port.) P. 39 N. 1580, Vila-de-Oleiros, prieuré de Crato du Castel-Branco. E. 16 déc. 1596, Coïmbre (Lus. 44, 143v). Emb. non prêtre, 4 avril 1600, sur le S. Vale

### Search for people with specific attribute

In [17]:
from sqlalchemy import select, and_
from sqlalchemy.orm import joinedload
from timelink.api.models import Person, Attribute

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).join(Person.attributes.and_(Attribute.the_type == 'jesuita-entrada',
                                                      Attribute.the_value.like('Coimbra')))
    print(stmt)
    persons = session.execute(stmt).scalars().all()

    for person in persons[:show_only]:
        print(person.to_kleio())


SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_source, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.extra_info, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id JOIN (entities AS entities_1 JOIN attributes AS attributes_1 ON attributes_1.id = entities_1.id) ON entities.id = attributes_1.entity AND attributes_1.the_type = :the_type_1 AND attributes_1.the_value LIKE :the_value_1
referido$António de Abreu/m/id=deh-antonio-de-abreu-ref1
  rel$function-in-act/referido/lista/dehergne-notices-a//obs=line: 20
  ls$nacionalidade/Portugal/
  ls$dehergne/1-ref1//obs=(Distinct du Provincial de Portugal de ce nom, 1627-1629 N. Lisbonne 1561, E. à Coïmbre 1576 (Lus. 43 II, 509v).
  ls$nascimento@wikidata/https://www.wikidata.org/wiki/Q597%Lisboa/1561
  ls$nascimento/Lisboa#@wikidata:Q597/1561
  ls$jesuita-entrada@wikidata/https://www.wiki

/var/folders/x7/qzytnd752559g0w475rfwkx00000gn/T/ipykernel_2685/3444709223.py:10: SAWarning: An alias is being generated automatically against joined entity Mapper[Attribute(attributes)] due to overlapping tables.  This is a legacy pattern which may be deprecated in a later release.  Use the aliased(<entity>, flat=True) construct explicitly, see the linked example. (Background on this warning at: https://sqlalche.me/e/20/xaj2)
  print(stmt)




###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [ ]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='Soure',
                    show_elements=['name'],
                    more_attributes=['faculdade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 140554 to 140533
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                4 non-null      object
 1   naturalidade.type   4 non-null      object
 2   naturalidade        4 non-null      object
 3   naturalidade.date   4 non-null      object
 4   naturalidade.line   4 non-null      int64 
 5   naturalidade.level  4 non-null      int64 
 6   naturalidade.obs    4 non-null      object
 7   faculdade           4 non-null      object
 8   faculdade.date      4 non-null      object
 9   faculdade.obs       4 non-null      object
 10  uc.entrada          4 non-null      object
 11  uc.entrada.date     4 non-null      object
 12  uc.entrada.obs      4 non-null      object
 13  uc.saida            4 non-null      object
 14  uc.saida.date       4 non-null      object
 15  uc.saida.obs        4 non-null      object
dtypes: int64(2), object(14)
m

In [ ]:
df.head(5)

,name,naturalidade,naturalidade.date,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs
id,,,,,,,,,,,,,
140554,António de Abreu,Soure,1681-10-01,None,Cânones,1681-10-01,None,1681-10-01,1681-10-01,None,1690-06-21,1690-06-21,None
140532,Anselmo Munhós de Abreu,Soure,1707-04-06,None,Medicina,1707-04-06,None,1707-04-06,1707-04-06,None,1719-01-01,1719-01-01,None
140712,Fernando Munhós de Abreu,Soure,1719-01-09,None,Cânones,1719-01-09,None,1719-01-09,1719-01-09,None,1722-10-17,1722-10-17,None
140533,Anselmo Munhós de Abreu,Soure,1737-10-01,None,Leis,1737-10-01,None,1737-10-01,1737-10-01,None,1740-10-01,1740-10-01,None




###  Remover colunas sem valores

---



###  Remove empty columns

In [ ]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 140554 to 140533
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   name                4 non-null      object
 1   naturalidade.type   4 non-null      object
 2   naturalidade        4 non-null      object
 3   naturalidade.date   4 non-null      object
 4   naturalidade.line   4 non-null      int64 
 5   naturalidade.level  4 non-null      int64 
 6   naturalidade.obs    4 non-null      object
 7   faculdade           4 non-null      object
 8   faculdade.date      4 non-null      object
 9   faculdade.obs       4 non-null      object
 10  uc.entrada          4 non-null      object
 11  uc.entrada.date     4 non-null      object
 12  uc.entrada.obs      4 non-null      object
 13  uc.saida            4 non-null      object
 14  uc.saida.date       4 non-null      object
 15  uc.saida.obs        4 non-null      object
dtypes: int64(2), object(14)
m

In [ ]:
df.head(5)

,name,naturalidade.type,naturalidade,naturalidade.date,naturalidade.line,naturalidade.level,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs
id,,,,,,,,,,,,,,,,
140554,António de Abreu,naturalidade,Soure,1681-10-01,12809,4,,Cânones,1681-10-01,,1681-10-01,1681-10-01,,1690-06-21,1690-06-21,
140532,Anselmo Munhós de Abreu,naturalidade,Soure,1707-04-06,11432,4,,Medicina,1707-04-06,,1707-04-06,1707-04-06,,1719-01-01,1719-01-01,
140712,Fernando Munhós de Abreu,naturalidade,Soure,1719-01-09,22020,4,,Cânones,1719-01-09,,1719-01-09,1719-01-09,,1722-10-17,1722-10-17,
140533,Anselmo Munhós de Abreu,naturalidade,Soure,1737-10-01,11505,4,,Leis,1737-10-01,,1737-10-01,1737-10-01,,1740-10-01,1740-10-01,



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [ ]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'faculdade' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc.entrada':'min',
                                                  'uc.saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

,id,uc.entrada,uc.saida
faculdade,,,
Cânones,2,1681-10-01,1722-10-17
Leis,1,1737-10-01,1740-10-01
Medicina,1,1707-04-06,1719-01-01



### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [ ]:
from timelink.pandas import attribute_values

df_totals = attribute_values('jesuita-entrada',
                             groupname='n',  # only in this Kleio group
                             db=tlnb.db)


In [ ]:
df_totals.head(10)


,count,date_min,date_max
value,,,
?,24,15760104,17581126
Coimbra,5,15420000,16770701
Goa,3,15480502,16660000
Lisboa,3,16680000,17460223
Avignon,2,17350731,17370927
Paris,2,16700926,16771029
Saragoça,2,16740613,16771115
Évora,2,15710304,17230000
Alcalá de Henares,1,15580413,15580413


#### Filtrar por datas

---

#### Filter by dates



In [ ]:
df_totals = attribute_values('grau',dates_between=('1535','1600'),db=tlnb.db)

In [ ]:
df_totals.head(10)

,count,date_min,date_max
value,,,
Bacharel,9,1554-07-19,1593-06-09
Formatura,6,1574-07-24,1593-05-12
Bacharel em Artes,3,1574-03-14,1588-03-12
Licenciado,3,1574-06-03,1597-05-06
Doutor,2,1560-12-22,1589-10-27
Licenciado em Artes,2,1574-05-15,1589-01-13


## Visualizar registos

---

## View records





### Search people by name, results in dataframe

In [ ]:
from timelink.pandas import pname_to_df
people = pname_to_df(name='%Abegão%',
                     db=tlnb.db,)
people.head()

,id,name,sex,obs
0,140338,Estevão de Matos Abegão,m,"""""""\n\n Id: 140338\n Cód..."
1,140339,Jerónimo Rodrigues Abegão,m,"""""""\n\n Id: 140339\n Cód..."
2,140339-pai,Jerónimo Rodrigues Abegão,m,


### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [ ]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = '140338'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf[['the_date','the_type','the_value','attr_obs']].sort_values(['the_date','the_type'])

,the_date,the_type,the_value,attr_obs
id,,,,
140338,1728-10-01,faculdade,Cânones,
140338,1728-10-01,faculdade.ano,Cânones.1728,
140338,1728-10-01,instituta,1728-10-01,01.10.1728 1728-10-01
140338,1728-10-01,instituta.ano,1728,01.10.1728 1728-10-01
140338,1728-10-01,naturalidade,Elvas,
140338,1728-10-01,naturalidade.ano,Elvas.1728,
140338,1728-10-01,nome,Estevão de Matos Abegão,
140338,1728-10-01,nome-geografico,Elvas,
140338,1728-10-01,nome-geografico.ano,Elvas.1728,


In [ ]:
p = tlnb.db.get_person(id)
print(p.to_kleio())

n$Estevão de Matos Abegão/m/id=140338/obs="""
      """

                  Id: 140338
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000002

                  Nome        : Estevão de Matos Abegão
                  Data inicial: 1728-10-01
                  Data final  : 1733-07-03
                  Filiação: Lourenço de Matos Pereira
                  Naturalidade: Elvas
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1729
                  01.10.1730
                  01.10.1732

                  Instituta: 01.10.1728

                  Bacharel: 03.07.1733
              """
  """
  rel$function-in-act/n/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000002""/2021-05-17
  atr$data-do-registo/2021-05-17/2021-05-17
  atr$url/""https://pesquisa.auc.uc.pt/details?id=140338""/2021-05-17
  ls$uc.entrada/1728-10-01/1728-10-01
  ls$uc.saida/1733-07-03/1733-07-03
  ls$uc.entrada.ano/1728/1728-1

#### Pessoas com determinados atributos, em colunas

Permite criar uma tabela com vários attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

Generate a table of people with attributes in columns

In [ ]:
# Get list of people with with a certain value in a specific attribute
from timelink.pandas import entities_with_attribute

id = 140338
# id = '316297'  # João Pedro Ribeiro
# id = '234295'  # Alexandre Metelo de Sousa
df = entities_with_attribute(
                    entity_type='person',
                    the_type='faculdade',  # we need a base attribute
                    the_value='Teologia',
                    show_elements=['name'],
                    more_attributes=['naturalidade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
view_cols = ['name','naturalidade','uc.entrada','faculdade','uc.saida','faculdade.obs']
df.info()
df[view_cols].sort_values(['name','uc.entrada'])

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 140719 to 269145
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               29 non-null     object
 1   faculdade.type     29 non-null     object
 2   faculdade          29 non-null     object
 3   faculdade.date     29 non-null     object
 4   faculdade.line     29 non-null     int64 
 5   faculdade.level    29 non-null     int64 
 6   faculdade.obs      29 non-null     object
 7   naturalidade       22 non-null     object
 8   naturalidade.date  22 non-null     object
 9   naturalidade.obs   22 non-null     object
 10  uc.entrada         29 non-null     object
 11  uc.entrada.date    29 non-null     object
 12  uc.entrada.obs     29 non-null     object
 13  uc.saida           29 non-null     object
 14  uc.saida.date      29 non-null     object
 15  uc.saida.obs       29 non-null     object
dtypes: int64(2), object(14)
memory usage: 4.9+

,name,naturalidade,uc.entrada,faculdade,uc.saida,faculdade.obs
id,,,,,,
140511,Abílio Ferreira Gomes de Abreu,"Cuenca, Coimbra",1854-10-02,Teologia,1856-07-09,"""Teologia#29"""
140516,Agostinho Machado de Abreu,Gesteira,1721-10-01,Teologia,1726-10-01,
140375,Alexandre de Brito e Sousa Abranches,Santa Ovaia,1863-10-12,Teologia,1867-06-12,
140534,Antão de Abreu,NaN,1638-07-20,Teologia,1639-11-20,
140350,António Camacho Guerreiro de Brito e Aboim,Mértola,1770-10-01,Teologia,1770-10-01,
140545,António Dinis de Abreu,Midões,1695-12-09,Teologia,1699-10-01,
140593,António Lopes Coelho de Abreu,Azóia,1861-10-11,Teologia,1864-06-20,
140552,António de Abreu,Sertã,1603-11-03,Teologia,1605-10-26,
140553,António de Abreu,Setúbal,1626-11-19,Teologia,1626-11-19,


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [ ]:
import pandas as pd
from timelink.pandas.group_attributes import display_group_attributes, group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome.apelido','nome.primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','matricula.ano','nome-geografico','instituta.ano',
         'uc.entrada.ano','uc.saida.ano']

dup_ids = ['140490','140503',]

display_group_attributes(dup_ids,
                            entity_type='person',
                            header_elements=['name'],
                            #header_attributes=['nome.nota','uc.entrada'],
                             exclude_attributes=no_show,
                             sort_attributes=['the_type','the_value','the_date',],
                             db=tlnb.db)

,name,the_type.type,the_type,the_type.date,the_type.line,the_type.level,the_type.obs,id
0,Manuel de Abrantes,uc.entrada,1705-10-01,1705-10-01,8851,4,,140490
1,Manuel de Abrantes,uc.entrada,1706-10-01,1706-10-01,9727,4,,140503


,name,the_type,the_value,the_date,attr_obs,id
0,Manuel de Abrantes,faculdade,Teologia,1705-10-01,,140490
1,Manuel de Abrantes,faculdade,Teologia,1706-10-01,,140503
2,Manuel de Abrantes,matricula,Teologia,1705-10-01,01.10.1705,140490
3,Manuel de Abrantes,matricula,Teologia,1706-10-01,01.10.1706,140503
4,Manuel de Abrantes,matricula,Teologia,1707-10-01,01.10.1707,140503
5,Manuel de Abrantes,matricula,Teologia,1708-10-01,01.10.1708,140503
6,Manuel de Abrantes,naturalidade,Vila de Matos,1706-10-01,,140503
7,Manuel de Abrantes,nome,Manuel de Abrantes,1705-10-01,,140490
8,Manuel de Abrantes,nome,Manuel de Abrantes,1706-10-01,,140503
9,Manuel de Abrantes,nome.nota,padre,1705-10-01,,140490
